In [3]:
## inputs that will be overriden into the pipe_vary_temp.py
## use run_sbatch_sequential to vary temp with respect to 1 conf file

import subprocess
import multiprocessing
import os, sys
import numpy as np


def generate_sbatch_sequential(steps, reps, conf_lammps_file, prefix, main_file, ntasks_per_node, start_temp, temp_reps, temp_index):
    
    filename = "temp-" + str(start_temp) + conf_lammps_file + ".sbatch"
    filename = filename.replace('..','').replace('/', '_')
    file = open(filename,"w+")
    
    file.write("""#!/bin/bash

#SBATCH --mail-type=ALL
#SBATCH --mail-user=mikejones@uchicago.edu

#SBATCH --job-name=3SPN2-dehybrid

##SBATCH --output=out.out
##SBATCH --partition=fela
##SBATCH --partition=broadwl
#SBATCH --partition=gm4-pmext
#SBATCH --nodes=1 # SET NUM NODES 
#SBATCH --gres=gpu:0 # SET NUM GPUS
#SBATCH --ntasks-per-node=""" + str(ntasks_per_node) + """ # SETS NUM MPI RANKS
#SBATCH --cpus-per-task=1 # SET NUM THREADS (cores)
#SBATCH --qos=gm4
##SBATCH --constraint=gold-6248

# source /home/mikejones/.bashrc
#/home/mikejones/scratch-midway2/test/lmp_serial -in DNA_with_GCGI_ions.in """)
               
    file.close()
     
    file = open(filename,"a+")  
    
    ## append_line = "\n\npython thread_from_sbatch.py " + str(temp) + " " + str(steps) + " " + str(reps) + " " + conf_lammps_file + " " + prefix + " " + main_file + " " + str(ntasks_per_node)
    
    append_line = "\n\npython thread_from_sbatch_sequentially.py " + str(ntasks_per_node) + " " + str(steps) + " " + str(reps) + " " + conf_lammps_file + " " + prefix + " " + main_file + " " + str(start_temp) + " " + str(temp_reps) + " " + str(temp_index) + " " + str(run_reps)
    
 
    file.write(append_line)
    file.close()
    
    return filename

def generate_sbatch_header(conf_lammps_file, ntasks_per_node, start_temp):
    
    filename = "temp-" + str(start_temp) + conf_lammps_file + ".sbatch"
    filename = filename.replace('..','').replace('/', '_')
    file = open(filename,"w+")
    
    file.write("""#!/bin/bash

#SBATCH --mail-type=ALL
#SBATCH --mail-user=mikejones@uchicago.edu

#SBATCH --job-name=3SPN2-dehybrid

##SBATCH --output=out.out
##SBATCH --partition=fela
##SBATCH --partition=broadwl
#SBATCH --partition=gm4-pmext
#SBATCH --nodes=1 # SET NUM NODES 
#SBATCH --gres=gpu:0 # SET NUM GPUS
#SBATCH --ntasks-per-node=""" + str(ntasks_per_node) + """ # SETS NUM MPI RANKS
#SBATCH --cpus-per-task=1 # SET NUM THREADS (cores)
#SBATCH --qos=gm4
##SBATCH --constraint=gold-6248

# source /home/mikejones/.bashrc
#/home/mikejones/scratch-midway2/test/lmp_serial -in DNA_with_GCGI_ions.in """)
               
    file.close()
    
    return filename

def append_sbatch_line(filename, steps, reps, conf_lammps_file, prefix, main_file, ntasks_per_node, start_temp, temp_reps, temp_index):
    
    file = open(filename,"a+")  
    
    append_line = "\n\npython thread_from_sbatch_sequentially.py " + str(ntasks_per_node) + " " + str(steps) + " " + str(reps) + " " + conf_lammps_file + " " + prefix + " " + main_file + " " + str(start_temp) + " " + str(temp_reps) + " " + str(temp_index) + " " + str(run_reps)
 
    file.write(append_line)
    file.close()


def run_sbatch():
    batch_list = []
    for temp in temps:
        for num in range(prefix_reps):
            
            
            prefix_num = prefix + str(num)
            batch_name = generate_sbatch(temp, steps, reps, conf_lammps_file, prefix_num, main_file, ntasks_per_node)
            subprocess.call(['sbatch', batch_name])

            
def run_sbatch_many_T():
    # iteratre through temps with prefix reps
    # no multiprocessing -- might not be necessary on fela
    
    batch_list = []
    for num in range(prefix_reps):
        
        temp = temps[num]     
        prefix_num = prefix + str(num)
        batch_name = generate_sbatch(temp, steps, reps, conf_lammps_file, prefix_num, main_file, ntasks_per_node)
        subprocess.call(['sbatch', batch_name]) 

        
def run_sbatch_sequential():
    
    batch_list = []
    sims = 0
    
    for num in range(prefix_reps):
        
        temp_0 = temp_start + temp_gap * int(sims / temp_reps)
        temp_index = sims % temp_reps
        
        prefix_num = prefix + str(num)
        batch_name = generate_sbatch_sequential(steps, reps, conf_lammps_file, prefix_num, main_file, ntasks_per_node, temp_0, temp_reps, temp_index)
        print(temp_0, temp_reps, temp_index)
        subprocess.call(['sbatch', batch_name]) 
        
        sims += ntasks_per_node ## should add 28 after each job

        
def run_sbatch_T_conf_dict():
    
    batch_list = []
    sims = 0
    
    for key in T_conf_dict:
        for conf in T_conf_dict[key]:
            
            temp_0 = key 
            temp_index = 0

            prefix_num = prefix + str(num)
            batch_name = generate_sbatch_sequential(steps, reps, conf, prefix_num, main_file, ntasks_per_node, temp_0, temp_reps, temp_index)
            print(temp_0, temp_reps, temp_index)
            subprocess.call(['sbatch', batch_name]) 

            sims += ntasks_per_node ## should add 28 after each job


In [8]:
## inputs that will be overriden into the pipe_vary_temp.py
## run at simulation temp and real temp?

reps  = 1                         # runs reps in series (should probs be changed)
prefix_reps = 3                   # number of seperate temperatures to run at
ntasks_per_node = 40               # number of parallel process (cpus) 28 to fill broadwl node
run_reps = 1                       # number of consequetive run commands (to bypass max runs)

temp_reps = ntasks_per_node        # number of sims to run at each temp

temp_gap = 0                       # seperation in Kelvin between each temp run      
steps = 50000000                 # 10^7 takes around 15 - 20 min, 10^8 takes 2 hours +

main_file = 'main_implicit.in'    #'main_implicit-240mM.in'
init_hybrid = True               # start from dissociated or hybridized state

seq = 'GC-core'                   # sequence 
temp_start = 314                 # low temp start

# use below commands for running automated sequence output under standard conditions
if init_hybrid:
    prefix = f'{seq}-implicit-hbr-'  #f'{seq}-implicit-240mM-'                  # name for output directory
    conf_lammps_file = f'../../seq_implicit_fix/{seq}/conf_lammps.in'  # initialization file read by main.in

else:
    prefix = f'{seq}-implicit-dis-'  #f'{seq}-implicit-240mM-'                  # name for output directory
    conf_lammps_file = f'../../seq_implicit_fix/{seq}/conf_lammps_diss.in'  # initialization file read by main.in

# mut uncomment in order to run
run_sbatch_sequential()

314 40 0
314 40 0
314 40 0


In [12]:
## runmany consequitives in the same job

## inputs that will be overriden into the pipe_vary_temp.py
## run at simulation temp and real temp?

reps  = 1                         # runs reps in series (should probs be changed)
prefix_reps = 3                   # number of jobs at fixed temp
ntasks_per_node = 40               # number of parallel process (cpus) 28 to fill broadwl node
run_reps = 1                       # number of consequetive run commands (to bypass max runs)

temp_reps = ntasks_per_node        

temp_gap = 5                    # seperation in Kelvin between each temp run   
ntemps   = 5                    # number of different temps to run at

steps = 50000000                 # 10^7 takes around 15 - 20 min, 10^8 takes 2 hours +

main_file = 'main_implicit.in'    #'main_implicit-240mM.in'
init_hybrid = True               # start from dissociated or hybridized state

seq = 'GC-end'                   # sequence 
temp_start = 313                  # low temp start


# use below commands for running automated sequence output under standard conditions
if init_hybrid:
    prefix = f'{seq}-implicit-hbr-'  #f'{seq}-implicit-240mM-'                  # name for output directory
    conf_lammps_file = f'../../seq_implicit_fix/{seq}/conf_lammps.in'  # initialization file read by main.in

else:
    prefix = f'{seq}-implicit-dis-'  #f'{seq}-implicit-240mM-'                  # name for output directory
    conf_lammps_file = f'../../seq_implicit_fix/{seq}/conf_lammps_diss.in'  # initialization file read by main.in

    
# make one header for multiple jobs
batch_name = generate_sbatch_header(conf_lammps_file, ntasks_per_node, temp_start)
    
batch_list = []
sims = 0

# iterate through per reps and add run command line by line
for i in range(ntemps):
    for num in range(prefix_reps):
   
        temp = temp_start + temp_gap*i
        temp_index = sims % temp_reps

        prefix_num = prefix + str(num)
        append_sbatch_line(batch_name, steps, reps, conf_lammps_file, prefix_num, main_file, ntasks_per_node, temp, temp_reps, temp_index)
        
        print(temp, num, temp_index, )
        sims += ntasks_per_node ## should add 28 after each job
    
# run job
subprocess.call(['sbatch', batch_name]) 

313 0 0
313 1 0
313 2 0
318 0 0
318 1 0
318 2 0
323 0 0
323 1 0
323 2 0
328 0 0
328 1 0
328 2 0
333 0 0
333 1 0
333 2 0


0